In [1]:
import pandas    as pd
import datetime as dt
import numpy as np
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,CategoricalEncoder,LabelEncoder

In [2]:
import pickle

In [4]:
with open('y.pickle','rb') as f:
    y = pickle.load(f)
    
with open('word_vetor.pickle','rb') as f:
    word_vector = pickle.load(f)
    
with open('test_data.pickle','rb') as f:
    test_data = pickle.load(f)

## train data

In [3]:
data = pd.read_csv(r'G:\DATASETS\Analytics Vidya\train_8wry4cB.csv')

In [4]:
data['start'] = data['startTime'].apply(lambda x:dt.datetime.strptime(x,"%d/%m/%y %H:%M"))
data['end'] = data['endTime'].apply(lambda x:dt.datetime.strptime(x,"%d/%m/%y %H:%M"))
data['session_time'] = data['end']-data['start']

def change_to_minutes(st):
    data_list = str(st).split(':')
    return int(data_list[1])+int(data_list[2])/60
data['duration'] = data['session_time'].apply(change_to_minutes)
data.drop(data.iloc[:,[0,1,2,5,6,7]],axis=1,inplace=True)

products = []
ind_prds = []
for dt in data['ProductList']:
    lst = dt.split(';')
    
    products.extend(lst)
    for lst2 in lst:
        sd = lst2.split('/')
        ind_prds.extend(sd)
    #print(dt)
for ls in ind_prds:
    if ls=='':
        ind_prds.remove('')   
        
y= data['gender']
word_dict = {}
for word in ind_prds:
    if word in word_dict:
        word_dict[word] +=1
    else:
        word_dict[word] = 1
word_final = dict(sorted(word_dict.items(),key=lambda x:x[1],reverse=True)[:3750])

vocab = {}
i=0 
for word in word_final.keys():
    if word in vocab:
        continue
    else:
        vocab[word] = i
        i +=1
#print(vocab)
total_list = []
for dt in data['ProductList']:
    lst = dt.split(';')
    inner_list = []
    for lst2 in lst:
        sd = lst2.split('/')
        inner_list.extend(sd)
    total_list.append(inner_list)
 

word_vector = []
for line in total_list:
    lexicon2 = [0]*len(vocab.keys())
    for word in line:
        
        if word in vocab:
            #print(vocab[word])
            
            lexicon2[vocab[word]] =1
            
    #print(lexicon2)
    word_vector.append(lexicon2)
for w,d in zip(word_vector,data['duration']):
    w.append(d)    

In [7]:
word_vector[0][3750]

1.0

## test data

In [10]:
test_data = pd.read_csv(r'G:\DATASETS\Analytics Vidya\test_Yix80N0.csv')

test_data['start'] = test_data['startTime'].apply(lambda x:dt.datetime.strptime(x,"%d/%m/%y %H:%M"))
test_data['end'] = test_data['endTime'].apply(lambda x:dt.datetime.strptime(x,"%d/%m/%y %H:%M"))
test_data['session_time'] = test_data['end']-test_data['start']
def change_to_minutes(st):
    data_list = str(st).split(':')
    return int(data_list[1])+int(data_list[2])/60
test_data['duration'] = test_data['session_time'].apply(change_to_minutes)
ID = test_data['session_id']
test_data.drop(test_data.iloc[:,[0,1,2,4,5,6]],axis=1,inplace=True)

test_list = []
for dt in test_data['ProductList']:
    lst = dt.split(';')
    inner_list = []
    for lst2 in lst:
        sd = lst2.split('/')
        inner_list.extend(sd)
    test_list.append(inner_list)
    
test_vector = []
for line in test_list:
    lexicon2 = [0]*len(vocab.keys())
    for word in line:
        
        if word in vocab:
            #print(vocab[word])
            
            lexicon2[vocab[word]] =1
            
    #print(lexicon2)
    test_vector.append(lexicon2)

for w,d in zip(test_vector,test_data['duration']):
    w.append(d)

In [17]:
len(test_data)
test_vector[0][3750]

0.0

In [5]:
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,BaggingClassifier,GradientBoostingClassifier,voting_classifier
from sklearn.metrics import accuracy_score

In [6]:
import pandas as pd
#submission = pd.read_csv(r'G:\DATASETS\Analytics Vidya\sample_submission_opxHi4g.csv')
#y_test = submission['gender']

In [7]:
logis = LogisticRegression()
logis.fit(word_vector,y)
predictions_logis = logis.predict(word_vector)
accuracy_score(logis.predict(word_vector),y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8943809523809524

In [23]:
predict = logis.predict(test_vector)
accuracy_score(predict,y_test)

0.5097777777777778

In [8]:
MNB = MultinomialNB()
MNB.fit(word_vector,y)
predictions_MNB = MNB.predict(word_vector)
accuracy_score(MNB.predict(word_vector),y)

0.8786666666666667

In [30]:
predict = MNB.predict(test_vector)
accuracy_score(predict,y_test)

0.5117777777777778

In [9]:
SGD = SGDClassifier()
SGD.fit(word_vector,y)
predictions_SGD = SGD.predict(word_vector)
accuracy_score(SGD.predict(word_vector),y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


0.8833333333333333

In [10]:
LSVC = LinearSVC()
LSVC.fit(word_vector,y)
predictions_LSVC = LSVC.predict(word_vector)
accuracy_score(LSVC.predict(word_vector),y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.9235238095238095

In [26]:
predict = LSVC.predict(test_vector)
accuracy_score(predict,y_test)

0.5055555555555555

In [16]:
RFC = RandomForestClassifier()
RFC.fit(word_vector,y)
predictions_RFC = RFC.predict(word_vector)
accuracy_score(RFC.predict(word_vector),y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9393333333333334

In [ ]:
with open('RFC.pickle','wb') as f:
    pickle.dump(RFC,f)

In [13]:
ABC = AdaBoostClassifier()
ABC.fit(word_vector,y)
predictions_ABC = ABC.predict(word_vector)
accuracy_score(ABC.predict(word_vector),y)

0.8783809523809524

In [14]:
bagging = BaggingClassifier()
bagging.fit(word_vector,y)
predictions_bagging = bagging.predict(word_vector)
accuracy_score(bagging.predict(word_vector),y)

0.94

In [15]:
GBC = GradientBoostingClassifier()
GBC.fit(word_vector,y)
predictions = GBC.predict(word_vector)
accuracy_score(GBC.predict(word_vector),y)

0.8792380952380953

In [31]:
from sklearn.model_selection import GridSearchCV,KFold,RandomizedSearchCV

In [42]:
param ={'n_estimators':[10,50,100],'criterion':['gini'] }

In [43]:
RZ = RandomizedSearchCV(estimator=RandomForestClassifier(),param_distributions=param,n_iter=10,cv=10)

In [ ]:
RZ.fit(word_vector,y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:271: UserWarning: The total space of parameters 3 is smaller than n_iter=10. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


In [ ]:
RZ_pred = RZ.predict(word_vector)
accuracy_score(RZ_pred,y)

In [33]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier()
KNN.fit(word_vector,y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [34]:
KNN_pred = KNN.predict(word_vector)
accuracy_score(KNN_pred,y)

0.8884761904761905

In [22]:
predict_data = pd.DataFrame({'predictions_bagging':predictions_bagging,
                             'predictions_LSVC':predictions_LSVC,'predictions_SGD':predictions_SGD,'predictions_logis':predictions_logis,
                            'predictions_RFC':predictions_RFC,'predictions_ABC':predictions_ABC,'predictions_MNB':predictions_MNB,})
from statistics import mode
result = []
for i in range(len(predict_data)):
    result.append(mode(predict_data.iloc[i]))

In [23]:
accuracy_score(result,y)

0.9213333333333333

In [24]:
predictions_bagging = bagging.predict(test_data)
predictions_RFC = RFC.predict(test_data)
predictions_LSVC = LSVC.predict(test_data)
predictions_logis =logis.predict(test_data)
predictions_SGD =SGD.predict(test_data)


predict_data = pd.DataFrame({'predictions_bagging':predictions_bagging,
                             'predictions_LSVC':predictions_LSVC,'predictions_SGD':predictions_SGD,'predictions_logis':predictions_logis,
                            'predictions_RFC':predictions_RFC})
from statistics import mode
result = []
for i in range(len(predict_data)):
    result.append(mode(predict_data.iloc[i]))

In [36]:
result[1:10]

['female',
 'female',
 'female',
 'female',
 'male',
 'male',
 'male',
 'female',
 'male']

In [27]:
t = pd.read_csv(r'G:\DATASETS\Analytics Vidya\test_Yix80N0.csv')
ID = t['session_id']


In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
print(confusion_matrix(predictions_RFC,y))
print(classification_report(predictions_RFC,y))

In [30]:
filename = r'G:\DATASETS\Analytics Vidya\submission_vote.csv'
submission = pd.DataFrame({'session_id':ID,'gender':result})
submission.to_csv(filename,index=False)